In [ ]:
import pandas as pd

# Cargar los datos
ventas_df = pd.read_csv('/workspace/ventas_peliculas.csv')
promocion_df = pd.read_csv('/workspace/promocion_peliculas.csv')

# Combinar los DataFrames usando un left join
merged_df = ventas_df.merge(promocion_df, on=['fecha', 'titulo'], how='left')

# Identificar valores nulos
print("Valores nulos en el dataset:")
print(merged_df.isna().sum())

# Rellenar valores nulos si es necesario
merged_df.fillna(0, inplace=True)

# Resumen estadístico de las ventas de boletos semanales
print("Resumen de ventas de boletos semanales:")
print(merged_df[['fecha', 'ventas_boletos']].groupby('fecha').sum())

# Comparación de semanas con y sin promociones
print("Ventas promedio con y sin promociones:")
print(merged_df.groupby(merged_df['eventos_promocionales'] > 0)['ventas_boletos'].mean())

# Detección de outliers
q1 = merged_df['ventas_boletos'].quantile(0.25)
q3 = merged_df['ventas_boletos'].quantile(0.75)
iqr = q3 - q1
outliers = merged_df[(merged_df['ventas_boletos'] < (q1 - 1.5 * iqr)) | (merged_df['ventas_boletos'] > (q3 + 1.5 * iqr))]
print("Valores extremos detectados en ventas de boletos:")
print(outliers)

# Análisis de correlación
correlation_matrix = merged_df[['ventas_boletos', 'calificacion', 'eventos_promocionales', 'participacion_promocion']].corr()
print("Matriz de correlación:")
print(correlation_matrix)

# Entrenar el modelo de regresión lineal
X = merged_df[['calificacion', 'eventos_promocionales', 'participacion_promocion']]
Y = merged_df['ventas_boletos']

# Calcular coeficientes de regresión manualmente
X = X.assign(constante=1)  # Agregar constante
beta = pd.Series(pd.DataFrame(X).T.dot(Y) / pd.DataFrame(X).T.dot(X).sum(), index=X.columns)
print("Coeficientes de regresión:")
print(beta)

# Predicciones y comparación
merged_df['ventas_predichas'] = X.dot(beta)
print("Comparación entre ventas reales y predichas:")
print(merged_df[['ventas_boletos', 'ventas_predichas']].head())